# Planet Viewing Example

This example shows how to plot geometric quantities using the frames system.


First we import numpy and the plotting module.

In [276]:
#%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

Then we import GODOT modules and create a universe from configuration.

The [universe](./universe.yml) contains an entry to insert a keplerian orbit point, named 'SC' into the frames system.

In [277]:
from godot.core import tempo, astro, events
from godot import cosmos
from godot.model import eventgen

# optionally avoid verbose logging messages
import godot.core.util as util
util.suppressLogger()

# create the universe
uni_config = cosmos.util.load_yaml('universe2.yml')
uni = cosmos.Universe(uni_config)

We setup the universe as defined in the yml file.
Furthermore we define a timespan, here one month, evaluated in 30 seconds timesteps.

In [278]:
#Setup
uni_config = cosmos.util.load_yaml('universe2.yml')
uni = cosmos.Universe(uni_config)

# specify a time grid
ep1 = tempo.Epoch('2026-04-01T00:00:00 TDB')
ep2 = tempo.Epoch('2026-05-01T00:00:00 TDB')
ran = tempo.EpochRange( ep1, ep2 )
grid = ran.createGrid(30.0) # 30 seconds stepsize
eps = 1e-6
tol = 1e-6
event_grid = ran.contract(eps).createGrid(30.0)

Events are evaluated as
$$
g(t) > 0
$$

Event interval are evaluated as time periods where this is true

Besides the earth hiding, we want to know wether the moon blocks the view of the spacecraft. This can be done by calculating if the vector between the ground station and the SC intersects the surface of the moon. If the vector does, and the spacecraft is closer to the groundstation than the moon, it is still visible

In [279]:
class PointProjector():
    def __init__(self,normal):
        self.normal = normal
        self.P = self.compute_projection_matrix(normal)
        self.u, self.v = self.find_basis_vectors(normal)  # Get 2D basis vectors

    def compute_projection_matrix(self, normal):
        """
        Computes the 3x3 projection matrix that projects a 3D point onto a plane 
        with the given normal vector.
        """
        normal = normal / np.linalg.norm(normal)  # Normalize the normal vector
        P = np.eye(3) - np.outer(normal, normal)  # Projection matrix
        return P

    def find_basis_vectors(self,normal):
        """
        Finds two orthogonal basis vectors in the plane perpendicular to the given normal.
        """
        normal = normal / np.linalg.norm(normal)
        
        # Pick an arbitrary vector not parallel to the normal
        if abs(normal[0]) < abs(normal[1]):
            u = np.array([1, 0, 0])
        else:
            u = np.array([0, 1, 0])
        
        # Make u orthogonal to normal
        u = u - np.dot(u, normal) * normal
        u = u / np.linalg.norm(u)
        
        # Compute v as cross product of normal and u
        v = np.cross(normal, u)
        v = v / np.linalg.norm(v)
        
        return u, v
    
    def change_basis(self, projected_point):
        x_2d = np.dot(projected_point, self.u)
        y_2d = np.dot(projected_point, self.v)
        arr_2d = np.array([x_2d, y_2d])
        return arr_2d

    def project_point(self, point):
        projected_point = self.P @ point  # Apply projection
        return projected_point

    def get_u_v_P(self):
        return self.u, self.v, self.P
    
    def plot_plane(self, ax, projected_3d):
        # Define plane grid
        normal = self.normal
        d = -np.dot(normal, projected_3d)
        xx, yy = np.meshgrid(np.linspace(-10, 10, 10), np.linspace(-10, 10, 10))
        zz = (-normal[0] * xx - normal[1] * yy - d) / normal[2]
        # Plot the plane
        ax.plot_surface(xx, yy, zz, color='cyan', alpha=0.5)

class Sphere():
    def __init__(self, coordinates, radius, points, name, color):
        """Helper function to plot a sphere."""
        self.centre = coordinates
        self.radius = radius
        u = np.linspace(0, 2 * np.pi, points)
        v = np.linspace(0, np.pi, points)
        self.x = coordinates[0] + radius * np.outer(np.cos(u), np.sin(v))
        self.y = coordinates[1] + radius * np.outer(np.sin(u), np.sin(v))
        self.z = coordinates[2] + radius * np.outer(np.ones(np.size(u)), np.cos(v))
        self.name = name
        self.color = color

    def plot_sphere(self, ax, alpha = 1.0):
        ax.plot_wireframe(self.x, self.y, self.z, alpha=alpha, label=self.name, color=self.color)

    def get_x_y_z(self):
        return self.x, self.y, self.z
    
    def set_2d(self, x, y):
        self.x2d = x
        self.y2d = y

In [280]:
import godot.core.astro as astro 
import godot.core.events as events
#Fungerer ikke helt med mine goals
import numpy as np

# Alias for Cebreros
groundStation = 'CB11'
def sphere_intersection(camera_pos, direction, radius = 1737.4):
    """Calculate the intersection of a vector from the camera position with a sphere."""
    direction = direction / np.linalg.norm(direction)  # Normalize direction
    b = 2 * np.dot(camera_pos, direction)
    c = np.dot(camera_pos, camera_pos) - radius**2
    discriminant = b**2 - 4 * c
    
    if discriminant < 0:
        return None  # No intersection
    t_1 = (-b - np.sqrt(discriminant))/2
    t_2 = (-b + np.sqrt(discriminant))/2

    t = t_1 if abs(t_1) < np.abs(t_2) else t_2
    intersection = camera_pos + t * direction
    return intersection

def CBSCnotBlockedByMoon( epo ):
    # Vector from moon to ground station
    GS = uni.frames.vector3('Moon', groundStation, 'ICRF', epo)
    # Vector from moon to space craft
    SC = uni.frames.vector3('Moon', 'SC', 'ICRF', epo)
    res = sphere_intersection(GS, GS-SC)
    if (res is None):
        return 1
    int_dist = np.sqrt(np.sum(np.power(GS - res,2)))
    sc_dist = np.sqrt(np.sum(np.power(GS-SC,2)))
    #print(f"Intersection distance: {int_dist}, spacecraft distance: {sc_dist}, closer?: {sc_dist < int_dist}")
    closer = sc_dist < int_dist
    if closer:
        return 1
    else:
        return -1

In [281]:
import matplotlib.pyplot as plt
def plot_scenario(t, ax, positions, celestial_objects):
    gsC, earthC, scC, moonC = positions

    # Visibility checks
    moonVisible = CBSCnotBlockedByMoon(t)
    model = uni.evaluables["Cebreros_Moon_Elevation"]
    elevation = np.degrees(model.eval(t))

    print(f"Time: {t} - Does moon block: {'Yes' if moonVisible < 0 else 'No'}, does earth block: {'Yes' if elevation < 10 else 'No'}")

    # Plot Sun
    celestial_objects['sun'].plot_sphere(ax, 0.25)
    celestial_objects['earth'].plot_sphere(ax, 0.25)
    celestial_objects['moon'].plot_sphere(ax, 0.25)
    celestial_objects['sc'].plot_sphere(ax, 1)
    celestial_objects['gs'].plot_sphere(ax, 1)

    # Plot vector from spacecraft to ground station
    vec = np.linspace(scC, earthC + gsC, 100)
    c = 'g' if moonVisible > 0 and elevation > 10 else 'r'
    ax.plot(vec[:, 0], vec[:, 1], vec[:, 2], label="SC to GS", color=c)
    
    ax.legend()

def get_celestial_objects(t, positions, do_print:bool):
    gsC, earthC, scC, moonC = positions
    suncentre = [0,0,0]

    # Visibility checks
    moonVisible = CBSCnotBlockedByMoon(t)
    model = uni.evaluables["Cebreros_Moon_Elevation"]
    elevation = np.degrees(model.eval(t))

    if do_print: print(f"Time: {t} - Does moon block: {'Yes' if moonVisible < 0 else 'No'}, does earth block: {'Yes' if elevation < 10 else 'No'}")

    # Plot Sun
    sun = Sphere(suncentre, 695500, 200, 'sun', 'orange')
    earth = Sphere(earthC, np.linalg.norm(gsC), 200, 'earth', color='b')
    moon = Sphere(moonC, 1737.4, 200, 'moon', color='y')

    # Plot spacecraft
    c = 'g' if moonVisible > 0  else 'r'
    spacecraft = Sphere(scC, 50, 200, 'sc', color=c)
    
    #Plot GroundStation
    c = 'g' if elevation > 10 else 'r'
    groundstation = Sphere(earthC + gsC, 50, 200, 'gs', color=c)

    return {'earth': earth, 'moon': moon, 'sc': spacecraft, 'gs': groundstation}


def set_view(earthcentre, mooncentre, ax, view):
    """Set axis limits based on the desired view."""
    vd_earth = 5000
    vd_moon = 2000
    if view == "Earth":
        ax.set_xlim([earthcentre[0]-vd_earth, earthcentre[0]+vd_earth])
        ax.set_ylim([earthcentre[1]-vd_earth, earthcentre[1]+vd_earth])
        ax.set_zlim([earthcentre[2]-vd_earth, earthcentre[2]+vd_earth])
    elif view == "Moon":
        ax.set_xlim([mooncentre[0]-vd_moon, mooncentre[0]+vd_moon])
        ax.set_ylim([mooncentre[1]-vd_moon, mooncentre[1]+vd_moon])
        ax.set_zlim([mooncentre[2]-vd_moon, mooncentre[2]+vd_moon])

def get_positions_at_time(t, uni, GS_name):
    gsC = uni.frames.vector3('Earth', GS_name, 'ICRF', t)
    earthC = uni.frames.vector3('Sun', 'Earth', 'ICRF', t)
    spacecraftC = uni.frames.vector3('Sun', 'SC', 'ICRF', t)
    moonC =  uni.frames.vector3('Sun', 'Moon', 'ICRF', t)
    return gsC, earthC, spacecraftC, moonC

In [299]:
import numpy as np
import godot

def do_plots(time:str, do_plot:bool, do_print:bool):
    t = godot.core.tempo.Epoch(time)
    focuses = ["Moon" , "", "Earth"]
    celestial_positions = get_positions_at_time(t, uni, 'CB11')
    _, earth_centre, _, moon_centre = celestial_positions
    celestial_objects = get_celestial_objects(t, celestial_positions, do_print)
    
    if do_plot:
        fig, ax = plt.subplots(1, 3, subplot_kw={'projection': '3d'}, figsize=(20,6))
        for i, focus in enumerate(focuses):
            plt.subplot(1,3,i+1)
            celestial_objects = plot_scenario(t, ax[i], celestial_positions, celestial_objects)
            set_view(earth_centre, moon_centre, ax[i], focus)
        plt.show()
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(111)
    normal =  uni.frames.vector3('Moon', 'Sun', 'ICRF', t)
    PP = PointProjector(normal)
    phi = np.linspace(0, 2*np.pi, 100)
    for celestial_object_name in celestial_objects:
        object = celestial_objects[celestial_object_name]
        cx, cy = PP.change_basis(PP.project_point(object.centre))
        object.set_2d(cx,cy)

    if do_plot:
        for celestial_object_name in celestial_objects:
            object = celestial_objects[celestial_object_name]
            circle = plt.Circle((cx,cy), object.radius, color=object.color, alpha = 0.5,label = f'{object.name}')
            ax.add_patch(circle)
        plt.axis('equal')
        plt.legend()
        plt.show()

    for celestial_object_name in celestial_objects:
        object = celestial_objects[celestial_object_name]
        if object.name == 'earth':
            earth = object
        if object.name == 'sc':
            sc = object

    is_sc_within_earth = np.power(earth.x2d - sc.x2d,2) + np.power(earth.y2d - sc.y2d, 2) < np.power(earth.radius, 2)
    if do_print: print('Is SC within earth?: ' + ('yes' if is_sc_within_earth else 'no'))
    if is_sc_within_earth: return 1
    else: return -1



In [302]:
#Setup
uni_config = cosmos.util.load_yaml('universe2.yml')
uni = cosmos.Universe(uni_config)

def bba(epo):
    return do_plots(epo, False, False)

# specify a time grid
ep1 = tempo.Epoch('2025-03-14T00:00:00 TDB')
ep2 = tempo.Epoch('2025-04-01T12:00:00 TDB')
ran = tempo.EpochRange( ep1, ep2 )
grid = ran.createGrid(30.0) # 30 seconds stepsize
eps = 1e-6
tol = 1e-6
event_grid = ran.contract(eps).createGrid(10.0)

CB_SC_visible = events.generateEventIntervalSet(bba, eps, grid, tol)

for interval in CB_SC_visible:
    start = interval.start().value()
    end = interval.end().value()
    print( f"View from {start} to {end}, duration {end - start} secs" )

KeyboardInterrupt: 